### Example Receipt:

<img src="http://www.trbimg.com/img-561b165a/turbine/la-sp-sarkisian-receipt-20151011" width="200">

### OCR using Tesseract
`$ brew install tesseract`

`$ pip install pytesseract`

In [1]:
from PIL import Image
import requests
from io import BytesIO
import pytesseract

In [2]:
img_url = "http://www.trbimg.com/img-561b165a/turbine/la-sp-sarkisian-receipt-20151011"
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))

In [3]:
ocr_text = pytesseract.image_to_string(img)
print (ocr_text)

THE SUNCADIA RESORT

TC GOLF HOUSE
1103 CLAIRE i"
V5 1275 6st 1
SARK

JUNOS'11 10: 16AM

35 HEINEKEN 157.50
35 COORS LT 175.00
12 GREY GOSSE 120.00
7 BUD LIGHT 35.00
7 BAJA CHICKEN 84.00
2 RANCHER 24.00
1 CLASSIC 8.00
1 SALMON BLT 13.00
1 DRIVER 12.00
6 CORONA 36.00
2 7-UP 4,50
Subtotal 669.00
Tax 53.52

3:36 Arent Que $722.52
FOR HOTEL GUEST ROOM CHARGE ONLY
Gratuity.


### Entity recognition using spacy
`$ pip install spacy`

`$ python -m spacy download en_core_web_sm`

In [4]:
import spacy

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
doc = nlp(ocr_text)
for entity in doc.ents:
    print(entity.text, entity.label_)

GOLF HOUSE ORG
1103 DATE
CLAIRE ORG

 GPE
1275 6st 1
 DATE
10 CARDINAL
35 CARDINAL
HEINEKEN ORG
157.50 PRODUCT
35 CARDINAL
COORS LT 175.00
12 GREY ORG
120.00 CARDINAL
7 BUD LIGHT PRODUCT
35.00 CARDINAL
84.00 CARDINAL

 GPE
2 CARDINAL
24.00 CARDINAL
8.00 CARDINAL

1 SALMON BLT 13.00
1 DRIVER EVENT
12.00 CARDINAL
6 CARDINAL
4,50
Subtotal 669.00
 ORG
722.52 MONEY

FOR HOTEL GUEST ROOM CHARGE ORG

 GPE


Techniques like NER (named entity recognition) could be used here, using tools like spacy
Spacy could be trained with custom entities, e.g. "item" "price" "total price" "account number"

### Some data cleaning & investigation

In [7]:
ocr_arr = ocr_text.split('\n')
ocr_arr

['THE SUNCADIA RESORT',
 '',
 'TC GOLF HOUSE',
 '1103 CLAIRE i"',
 'V5 1275 6st 1',
 'SARK',
 '',
 "JUNOS'11 10: 16AM",
 '',
 '35 HEINEKEN 157.50',
 '35 COORS LT 175.00',
 '12 GREY GOSSE 120.00',
 '7 BUD LIGHT 35.00',
 '7 BAJA CHICKEN 84.00',
 '2 RANCHER 24.00',
 '1 CLASSIC 8.00',
 '1 SALMON BLT 13.00',
 '1 DRIVER 12.00',
 '6 CORONA 36.00',
 '2 7-UP 4,50',
 'Subtotal 669.00',
 'Tax 53.52',
 '',
 '3:36 Arent Que $722.52',
 'FOR HOTEL GUEST ROOM CHARGE ONLY',
 'Gratuity.']

Getting every line ending with a number:

In [8]:
def valid_line(s):
    str_arr = s.split(' ')
    return (str_arr[len(str_arr) - 1].replace(".", "", 1).isdigit())

def number_ending_lines(items):
    return list(filter(lambda x: valid_line(x), items))

In [9]:
filtered_list = number_ending_lines(ocr_arr)
filtered_list

['V5 1275 6st 1',
 '35 HEINEKEN 157.50',
 '35 COORS LT 175.00',
 '12 GREY GOSSE 120.00',
 '7 BUD LIGHT 35.00',
 '7 BAJA CHICKEN 84.00',
 '2 RANCHER 24.00',
 '1 CLASSIC 8.00',
 '1 SALMON BLT 13.00',
 '1 DRIVER 12.00',
 '6 CORONA 36.00',
 'Subtotal 669.00',
 'Tax 53.52']

Getting the number at the end of the line with the word 'total' in it

In [10]:
def get_total(arr):
    total = 0
    for item in arr:
        if ("total" in item.lower()):
            return float(item.split(' ')[len(item.split(' ')) - 1])
    return 0

In [11]:
total = get_total(filtered_list)
total

669.0

Trying to use subset sum on all the numbers at the end of lines to sum to the total. e.g. the items purchased

In [12]:
def subsetsum(array,num):

    if num == 0 or num < 1:
        return None
    elif len(array) == 0:
        return None
    else:
        if array[0] == num:
            return [array[0]]
        else:
            with_v = subsetsum(array[1:],(num - array[0])) 
            if with_v:
                return [array[0]] + with_v
            else:
                return subsetsum(array[1:],num)
def get_prices(items):
    prices = []
    for item in items:
        prices.append(float(item.split(' ')[len(item.split(' ')) - 1]))
    prices = list(filter(lambda x: x != total, prices))
    return prices

In [17]:
prices = get_prices(filtered_list)
prices

[1.0, 157.5, 175.0, 120.0, 35.0, 84.0, 24.0, 8.0, 13.0, 12.0, 36.0, 53.52]

In [18]:
ans = subsetsum(prices, total)

In [19]:
print (ans)

None


The '2 7-UP 4,50' line was ignored as the decimal point was read as a comma

In [20]:
prices.append(4.50)
ans_with_7up = subsetsum(prices, total)
print (ans_with_7up)

[157.5, 175.0, 120.0, 35.0, 84.0, 24.0, 8.0, 13.0, 12.0, 36.0, 4.5]


Adding the 4.50 back gives us a valid subset of our original set - showing which numbers represent subpurchases on this receipt